<a href="https://colab.research.google.com/github/cyberfury/Kaggle/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
test_path="/content/drive/MyDrive/KaggleDownload/titanic/test.csv"
train_path="/content/drive/MyDrive/KaggleDownload/titanic/train.csv"

test_df=pd.read_csv(test_path)
train_df=pd.read_csv(train_path)


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
# 분류 문제 
# 로지스틱 / xg부스트 / 앙상블 사용해볼것
# 각 데이터를 수치형으로 변환
# 상관성이있는 데이터들을 추출
# 학습 후 예측

In [12]:
# 추후 test 데이터도 전처리 해줘야 하기에, 카피 데이터프레임을 생성해둔다
df=train_df.copy() 

In [13]:
# Cabin, Ticket, Name 값은 100개이상으로 숫자변환 대상에서 우선 제외
# Pclass, SibSp, Parch, Fare 이미 숫자로 나타나 있어 굳이 고칠필요없음
# Age는 상관성이 높은 Pclass,SibSp 을 학습시켜 대입해볼것 - 상관성 0.3이상

###################################################################
# Embarked, Sex는 아래처럼 전처리함

# 전처리1. 성별을 숫자로 변환
# 성별은 male, female 바이너리 값임
df['Sex']=df['Sex'].apply(lambda x: 0 if x=='male' else 1)

# 전처리2. Embarked 을 원핫인코딩함. 
# Embarked 는 S C Q 문자로 나타남
df_merged=pd.concat([df,pd.get_dummies(df['Embarked'])],axis=1)
###################################################################

# 최종 Feature
# [['Sex','Embarked','SibSp','Pclass','Fare','Parch']]

In [30]:
# Age의 빈값을, Pclass, SibSP 값으로 예측값을 넣어보자

import sklearn 
from sklearn import *

# 모델분류
# sklearn.model_selection.train_test_split
# 학습모델 - 랜덤포레스트
# sklearn.ensemble.RandomForestClassifier

In [53]:
# Age가 존재하는 데이터 = 학습데이터, Age가 없는 데이터 = 학습모델로 Age를 구해볼 데이터
df_age_model=df_merged[df_merged['Age'].notnull()][['PassengerId','Pclass','SibSp','Age']].copy() # Age가 존재하는 데이터로, 학습데이터셋을 구성

In [54]:
# 학습/검증 데이터셋 분류, 지도 학습 임으로
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df_age_model[['Pclass','SibSp']], df_age_model['Age'], test_size=0.33, random_state=714)

In [ ]:
# 상관계수는 0.3정도인데, 선형성을 띄느지 확인 필요
[['PassengerId','Pclass','SibSp','Age']]. >> 산점도 그려서 회귀할지, 분류모델할지 결정 필요

In [18]:
df_merged

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,S,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,S,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,S,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,C,1,0,0


In [26]:
# 상관성 현황
df_merged.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
PassengerId,1.000000,-0.005007,-0.035144,NaN,0.036847,-0.057527,-0.001652,0.012658,-0.001205,-0.033606,0.022148
Survived,-0.005007,1.000000,-0.338481,NaN,-0.077221,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.155660
Pclass,-0.035144,-0.338481,1.000000,NaN,-0.369226,0.083081,0.018443,-0.549500,-0.243292,0.221009,0.081720
Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,0.036847,-0.077221,-0.369226,NaN,1.000000,-0.308247,-0.189119,0.096067,0.036261,-0.022405,-0.032523
SibSp,-0.057527,-0.035322,0.083081,NaN,-0.308247,1.000000,0.414838,0.159651,-0.059528,-0.026354,0.070941
Parch,-0.001652,0.081629,0.018443,NaN,-0.189119,0.414838,1.000000,0.216225,-0.011069,-0.081228,0.063036
Fare,0.012658,0.257307,-0.549500,NaN,0.096067,0.159651,0.216225,1.000000,0.269335,-0.117216,-0.166603
C,-0.001205,0.168240,-0.243292,NaN,0.036261,-0.059528,-0.011069,0.269335,1.000000,-0.148258,-0.778359
Q,-0.033606,0.003650,0.221009,NaN,-0.022405,-0.026354,-0.081228,-0.117216,-0.148258,1.000000,-0.496624
